In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230227'
and yyyymmdd <= '20230302'
and city_name in ('Chennai')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,63fc29bd62511300abbee505,2.846,88618c4749fffff,12.842419,80.226837,Chennai,88618c559dfffff,12.806330,88618c559dfffff,12.806236,80.220802,88618c4749fffff,12.842419,80.226837,80.220690,092541,09,0915,Link,09:00,1,20230227,dropped,None
1,63fc2a2dbdfb0a23d5940d6a,NaN,88618c45a9fffff,12.932900,80.123444,Chennai,88618c45a9fffff,12.932900,88618c45a9fffff,12.932928,80.123448,88618c4487fffff,12.871034,80.076589,80.123444,092733,09,0915,Link,09:00,1,20230227,customerCancelled,order cancelled before rider accepted
2,63fc2a568acbc800ee74d63d,1.867,88618c4c37fffff,13.049579,80.155502,Chennai,88618c4dd9fffff,13.045832,88618c4dd9fffff,13.045897,80.144345,88618c4c37fffff,13.049579,80.155502,80.144463,092814,09,0915,Link,09:00,1,20230227,dropped,None
3,63fc2a8401136245f3aa7979,2.335,88618c4ca7fffff,13.059084,80.113716,Chennai,88618c4d9bfffff,13.068480,88618c4d9bfffff,13.068721,80.120895,88618c4ca7fffff,13.059084,80.113716,80.121056,092900,09,0915,Link,09:00,1,20230227,dropped,Other
4,63fc2ab32c52d456af3887b9,0.459,88618c4ce7fffff,13.039551,80.131788,Chennai,88618c4ce7fffff,13.043446,88618c4ce7fffff,13.039579,80.131656,88618c4c07fffff,13.030898,80.168123,80.130508,092947,09,0915,Link,09:00,1,20230227,customerCancelled,customer not reachable


In [5]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,63fc29bd62511300abbee505,2.846,88618c4749fffff,12.842419,80.226837,Chennai,88618c559dfffff,12.806330,88618c559dfffff,12.806236,80.220802,88618c4749fffff,12.842419,80.226837,80.220690,092541,09,0915,Link,09:00,1,20230227,dropped,None,41,25,27,1
1,63fc2a2dbdfb0a23d5940d6a,NaN,88618c45a9fffff,12.932900,80.123444,Chennai,88618c45a9fffff,12.932900,88618c45a9fffff,12.932928,80.123448,88618c4487fffff,12.871034,80.076589,80.123444,092733,09,0915,Link,09:00,1,20230227,customerCancelled,order cancelled before rider accepted,33,27,27,1
2,63fc2a568acbc800ee74d63d,1.867,88618c4c37fffff,13.049579,80.155502,Chennai,88618c4dd9fffff,13.045832,88618c4dd9fffff,13.045897,80.144345,88618c4c37fffff,13.049579,80.155502,80.144463,092814,09,0915,Link,09:00,1,20230227,dropped,None,14,28,27,1
3,63fc2a8401136245f3aa7979,2.335,88618c4ca7fffff,13.059084,80.113716,Chennai,88618c4d9bfffff,13.068480,88618c4d9bfffff,13.068721,80.120895,88618c4ca7fffff,13.059084,80.113716,80.121056,092900,09,0915,Link,09:00,1,20230227,dropped,Other,00,29,27,1
4,63fc2ab32c52d456af3887b9,0.459,88618c4ce7fffff,13.039551,80.131788,Chennai,88618c4ce7fffff,13.043446,88618c4ce7fffff,13.039579,80.131656,88618c4c07fffff,13.030898,80.168123,80.130508,092947,09,0915,Link,09:00,1,20230227,customerCancelled,customer not reachable,47,29,27,1


In [6]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Chennai')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [7]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,63fc29bd62511300abbee505,2.846,88618c4749fffff,12.842419,80.226837,Chennai,88618c559dfffff,12.806330,88618c559dfffff,12.806236,80.220802,88618c4749fffff,12.842419,80.226837,80.220690,092541,09,0915,Link,09:00,1,20230227,dropped,None,41,25,27,1,CHN KELAMBAKKAM,Siruseri
1,63fc2a2dbdfb0a23d5940d6a,NaN,88618c45a9fffff,12.932900,80.123444,Chennai,88618c45a9fffff,12.932900,88618c45a9fffff,12.932928,80.123448,88618c4487fffff,12.871034,80.076589,80.123444,092733,09,0915,Link,09:00,1,20230227,customerCancelled,order cancelled before rider accepted,33,27,27,1,West Tambaram,CHN VANDALUR
2,63fc2a568acbc800ee74d63d,1.867,88618c4c37fffff,13.049579,80.155502,Chennai,88618c4dd9fffff,13.045832,88618c4dd9fffff,13.045897,80.144345,88618c4c37fffff,13.049579,80.155502,80.144463,092814,09,0915,Link,09:00,1,20230227,dropped,None,14,28,27,1,Vanagaram,Vanagaram
3,63fc2a8401136245f3aa7979,2.335,88618c4ca7fffff,13.059084,80.113716,Chennai,88618c4d9bfffff,13.068480,88618c4d9bfffff,13.068721,80.120895,88618c4ca7fffff,13.059084,80.113716,80.121056,092900,09,0915,Link,09:00,1,20230227,dropped,Other,00,29,27,1,Thiruverkadu,Kattupakkam
4,63fc2ab32c52d456af3887b9,0.459,88618c4ce7fffff,13.039551,80.131788,Chennai,88618c4ce7fffff,13.043446,88618c4ce7fffff,13.039579,80.131656,88618c4c07fffff,13.030898,80.168123,80.130508,092947,09,0915,Link,09:00,1,20230227,customerCancelled,customer not reachable,47,29,27,1,Kattupakkam,Porur


In [8]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_cluster'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_cluster'])['order_id'].nunique().reset_index()
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,order_id


In [21]:
cluster_grouped = df_morning.groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped = cluster_grouped.sort_values(by = 'order_id', ascending=False)
cluster_grouped.head(20)

,pickup_cluster,order_id
90,Nungambakkam,3929
127,T Nagar,3518
156,kilpauk,3179
84,Mylapore,2147
12,Anna Nagar,2057
133,Thiruvanmiyur,1935
17,Ashok Nagar Chennai,1868
101,Perambur,1859
122,Shenoy Nagar,1854
130,Teynampet,1811


In [9]:
#Query data for online captains

q_online_captains = """
select
        yyyymmdd, substr(hhmm, 1, 2) as hour, cluster,
        count(distinct captain_id) as Online_Captains
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230227'
        and yyyymmdd <= '20230302'
        and city in ('Chennai')
        and a.resolution = '8' 
        and service in ('Link')
        and substr(hhmm, 1, 2) in ('08', '09', '10', '11')
)
group by 1,2,3
"""

#Load data into pandas table
df_online_captains_copy = pd.read_sql(q_online_captains, connection)

In [10]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230227,11,Alwarpet,133
1,20230301,08,Perungudi,17
2,20230302,09,Kotturpuram,75
3,20230227,10,Mogappair East,90
4,20230227,10,Alwarpet,127


In [11]:
df_online_captains.groupby(['yyyymmdd', 'hour'])['Online_Captains'].sum().reset_index()

,yyyymmdd,hour,Online_Captains
0,20230227,08,9014
1,20230227,09,11453
2,20230227,10,13336
3,20230227,11,14528
4,20230228,08,9442
5,20230228,09,12446
6,20230228,10,14195
7,20230228,11,15014
8,20230301,08,9299
9,20230301,09,12014


In [12]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230227,11,Alwarpet,133
1,20230301,08,Perungudi,17
2,20230302,09,Kotturpuram,75
3,20230227,10,Mogappair East,90
4,20230227,10,Alwarpet,127


In [13]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [14]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
0,63fc29bd62511300abbee505,2.846,88618c4749fffff,12.842419,80.226837,Chennai,88618c559dfffff,12.806330,88618c559dfffff,12.806236,80.220802,88618c4749fffff,12.842419,80.226837,80.220690,092541,09,0915,Link,09:00,1,20230227,dropped,None,41,25,27,1,CHN KELAMBAKKAM,Siruseri,47.0,66.0,0.7121
1,63fc2a2dbdfb0a23d5940d6a,NaN,88618c45a9fffff,12.932900,80.123444,Chennai,88618c45a9fffff,12.932900,88618c45a9fffff,12.932928,80.123448,88618c4487fffff,12.871034,80.076589,80.123444,092733,09,0915,Link,09:00,1,20230227,customerCancelled,order cancelled before rider accepted,33,27,27,1,West Tambaram,CHN VANDALUR,37.0,105.0,0.3524
2,63fc2a568acbc800ee74d63d,1.867,88618c4c37fffff,13.049579,80.155502,Chennai,88618c4dd9fffff,13.045832,88618c4dd9fffff,13.045897,80.144345,88618c4c37fffff,13.049579,80.155502,80.144463,092814,09,0915,Link,09:00,1,20230227,dropped,None,14,28,27,1,Vanagaram,Vanagaram,25.0,73.0,0.3425
3,63fc2a8401136245f3aa7979,2.335,88618c4ca7fffff,13.059084,80.113716,Chennai,88618c4d9bfffff,13.068480,88618c4d9bfffff,13.068721,80.120895,88618c4ca7fffff,13.059084,80.113716,80.121056,092900,09,0915,Link,09:00,1,20230227,dropped,Other,00,29,27,1,Thiruverkadu,Kattupakkam,35.0,84.0,0.4167
4,63fc2ab32c52d456af3887b9,0.459,88618c4ce7fffff,13.039551,80.131788,Chennai,88618c4ce7fffff,13.043446,88618c4ce7fffff,13.039579,80.131656,88618c4c07fffff,13.030898,80.168123,80.130508,092947,09,0915,Link,09:00,1,20230227,customerCancelled,customer not reachable,47,29,27,1,Kattupakkam,Porur,57.0,97.0,0.5876


In [15]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio,weekday_name
0,63fc29bd62511300abbee505,2.846,88618c4749fffff,12.842419,80.226837,Chennai,88618c559dfffff,12.806330,88618c559dfffff,12.806236,80.220802,88618c4749fffff,12.842419,80.226837,80.220690,092541,09,0915,Link,09:00,1,20230227,dropped,None,41,25,27,1,CHN KELAMBAKKAM,Siruseri,47.0,66.0,0.7121,Monday
1,63fc2a2dbdfb0a23d5940d6a,NaN,88618c45a9fffff,12.932900,80.123444,Chennai,88618c45a9fffff,12.932900,88618c45a9fffff,12.932928,80.123448,88618c4487fffff,12.871034,80.076589,80.123444,092733,09,0915,Link,09:00,1,20230227,customerCancelled,order cancelled before rider accepted,33,27,27,1,West Tambaram,CHN VANDALUR,37.0,105.0,0.3524,Monday
2,63fc2a568acbc800ee74d63d,1.867,88618c4c37fffff,13.049579,80.155502,Chennai,88618c4dd9fffff,13.045832,88618c4dd9fffff,13.045897,80.144345,88618c4c37fffff,13.049579,80.155502,80.144463,092814,09,0915,Link,09:00,1,20230227,dropped,None,14,28,27,1,Vanagaram,Vanagaram,25.0,73.0,0.3425,Monday
3,63fc2a8401136245f3aa7979,2.335,88618c4ca7fffff,13.059084,80.113716,Chennai,88618c4d9bfffff,13.068480,88618c4d9bfffff,13.068721,80.120895,88618c4ca7fffff,13.059084,80.113716,80.121056,092900,09,0915,Link,09:00,1,20230227,dropped,Other,00,29,27,1,Thiruverkadu,Kattupakkam,35.0,84.0,0.4167,Monday
4,63fc2ab32c52d456af3887b9,0.459,88618c4ce7fffff,13.039551,80.131788,Chennai,88618c4ce7fffff,13.043446,88618c4ce7fffff,13.039579,80.131656,88618c4c07fffff,13.030898,80.168123,80.130508,092947,09,0915,Link,09:00,1,20230227,customerCancelled,customer not reachable,47,29,27,1,Kattupakkam,Porur,57.0,97.0,0.5876,Monday


In [16]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [17]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [18]:
df_morning_filter.to_csv('chennai_demand_supply_morning.csv', index = False)